#### Make sure this is uploaded onto GitHub
Put all needed libraries into a env.yml file
Show them how to download conda and clone repo into their computer
#### OR i can put this on an azure VM

#### Prerequisities: DEM, mangrove/non-mangrove layers
Python 3.6+, GIS, Anaconda, and following packages
#### Output: biomass raster 

In [10]:
# packages needed
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import json
import os
from os.path import dirname, abspath
from pathlib import Path

#### Step 1: Isolate mangrove classes from mangrove/nonmangrove layer

In [2]:
# helper function to get mangrove polygons out of input shapefile
def get_mangrove(label):
    # read file in with gpd
    polygons = gpd.read_file(label)
    # subset mangrove polygons
    mangrove = polygons[polygons['Class']=='mangrove']
    # initialize crs- this can be reprojected later
    mangrove.crs = {'init': 'epsg:4326'}
    # return dataframe
    return(mangrove)

#### Step 2: Get mangrove extent from DEM and calculate biomass 

In [3]:
# clip dem from vector mangrove shapefile- geopandas/rasterio version
def clip_dem2biomass(dest, dem, label):
    # check if file already exists, delete it
    if os.path.isfile(dest):
        os.remove(dest)
    # open files
    dem_ = rasterio.open(dem)
    # format for rasterio
    label_ = [json.loads(label.to_json())['features'][0]['geometry']]
    
    # print progress of the clipping
    print('Clipping...')
    out_img, out_transform = mask(dataset=dem_, shapes=label_, crop=True)
    print('Done!')
    
    # copy over metadata
    out_meta = dem_.meta
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform,
                    })
    
    # calculate biomass
    nodata = dem_.nodata
    bio_img = 10.8*(out_img)+35
    # replace nodata values back
    bio_img[bio_img == np.min(bio_img)] = nodata
    
    # writes dem out to file
    with rasterio.open(dest, "w", **out_meta) as dest:
        dest.write(bio_img)
    return

In [4]:
# # example block
# lap_1_2018_dem = "/data/mangrove-data/master/2018.07-mexico-lapaz/lap-2018-07-site1/dem/lap_2018-07_site1_120m_DEM_tc.tif"
# lap1_2018_shp = "/data/mangrove-data/Kathy/biomass_project/sites/lap_07-18_site1/lap_2018-07_site01_labels_m-nm/lap_2018-07_site01_labels_m-nm.shp"
# temp_dest = "/data/mangrove-data/Kathy/biomass_project/downsampling/lap_07-18_site1/test_mask.tif"

# # get mangrove polygons only
# m_poly = get_mangrove(lap1_2018_shp)
# # clip dems
# clip_dem2biomass(temp_dest, lap_1_2018_dem, m_poly)

In [ ]:
## Some example files to run with
# DEM: /data/mangrove-data/master/2018.07-mexico-lapaz/lap-2018-07-site1/dem/lap_2018-07_site1_120m_DEM_tc.tif
# shp: /data/mangrove-data/Kathy/biomass_project/sites/lap_07-18_site1/lap_2018-07_site01_labels_m-nm/lap_2018-07_site01_labels_m-nm.shp
# dest /data/mangrove-data/Kathy/biomass_project/downsampling/lap_07-18_site1/test_mask.tif

In [17]:
# Prompt user for manual or automated argument
while (True):
    # ask for dem file path
    dem = str(input("Please enter the full path to your DEM raster file: "))
    if (os.path.isfile(dem) == False):
        print('Invalid file path')
        continue
    # ask for label shpaefile path
    shp = str(input("Please enter the full path to your mangrove/nonmangrove shapefile: "))
    if (os.path.isfile(shp) == False):
        print('Invalid file path')
        continue
    # ask for destination path
    dest = str(input("Please enter the full path for your output biomass raster file (destination): "))
    if (os.path.isdir(dirname(dest)) == False):
        print('Invalid file path')
        continue
    break

Please enter the full path to your DEM raster file: /data/mangrove-data/master/2018.07-mexico-lapaz/lap-2018-07-site1/dem/lap_2018-07_site1_120m_DEM_tc.tif
Please enter the full path to your mangrove/nonmangrove shapefile: /data/mangrove-data/Kathy/biomass_project/sites/lap_07-18_site1/lap_2018-07_site01_labels_m-nm/lap_2018-07_site01_labels_m-nm.shp
Please enter the full path for your output biomass raster file (destination): /data/mangrove-data/Kathy/biomass_project/downsampling/lap_07-18_site1/test_mask.tif


In [18]:
## run code
# get mangrove polygons only
m_poly = get_mangrove(shp)
# clip dems
clip_dem2biomass(dest, dem, m_poly)

Clipping...
Done!
